# Pedestrian detection (Haar cascades) pedestrian2.mp4

In [1]:
# OpenCV Python program to detect cars in video frame 
# import libraries of python OpenCV  
import cv2 
import os
import time
from timeit import default_timer as timer


# path setting
common_path = 'C:\\Users\\yukir\\anaconda3\\envs\\opencv\\Lib\\site-packages\\cv2\\data'
pedestrianPath='C:\\Users\\yukir\\workspace\\opencv_work\\Vehicle-And-Pedestrian-Detection-Using-Haar-Cascades\\Main Project\\Main Project\\Pedestrian Detection'

movie_path = os.path.join(pedestrianPath,"pedestrian2.mp4")

# capture frames from a video 
cap = cv2.VideoCapture(movie_path) 
start_frame=16000
bgs = cv2.createBackgroundSubtractorKNN()

if not cap.isOpened():
    raise IOError(("Couldn't open video file or webcam. If you're "
            "trying to open a webcam, make sure you video_path is an integer!"))
    
# Trained XML classifiers describes some features of some object we want to detect 

common_path = 'C:\\Users\\yukir\\anaconda3\\envs\\opencv\\Lib\\site-packages\\cv2\\data'
pedestrianPath='C:\\Users\\yukir\\workspace\\opencv_work\\Vehicle-And-Pedestrian-Detection-Using-Haar-Cascades\\Main Project\\Main Project\\Pedestrian Detection'

person_cascade = cv2.CascadeClassifier(os.path.join(common_path,'haarcascade_fullbody.xml')) 
#person_cascade = cv2.CascadeClassifier(os.path.join(pedestrianPath,'pedestrian.xml')) 
#car_cascade = cv2.CascadeClassifier(os.path.join(common_path,'cars.xml'))
  
# Skip frames until reaching start_frame
if start_frame > 0:
        cap.set(cv2.CAP_PROP_POS_MSEC, start_frame)
            
accum_time = 0
curr_fps = 0
fps = "FPS: ??"
prev_time = timer()    
    
# loop runs if capturing has been initialized. 
count_num=0
while True: 
    # reads frames from a video 
    ret, frames = cap.read() 
      
    # convert to gray scale of each frames 
    gray = cv2.cvtColor(frames, cv2.COLOR_BGR2GRAY) 
    #mask = bgs.apply(frames)
    #bg = bgs.getBackgroundImage()  
  
    # Detects cars of different sizes in the input image 
    people = person_cascade.detectMultiScale(frames, scaleFactor=1.1, minNeighbors=2, minSize=(30, 30))       
    
    # To draw a rectangle
    for (x,y,w,h) in people: 
        cv2.rectangle(frames,(x,y),(x+w,y+h),(255,255,255),2) 

        text = "person" 
        text_top = (x, y-10) 
        text_bot = (x + 50, y + 5)
        text_pos = (x + 5, y)
        cv2.rectangle(frames, text_top, text_bot,(255,255,255), -1)
        cv2.putText(frames, text, text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,0), 1)  
        
        
    # Calculate FPS
    # This computes FPS for everything, not just the model's execution 
    # which may or may not be what you want
    curr_time = timer()
    exec_time = curr_time - prev_time
    prev_time = curr_time
    accum_time = accum_time + exec_time
    curr_fps = curr_fps + 1
    count_num = count_num + 1
    if accum_time > 0:
        accum_time = accum_time - 1
        fps = "FPS: " + str(curr_fps)
        curr_fps = 0        
        counts = "frame_count: " + str(count_num)
    # Draw FPS in top left corner
    cv2.rectangle(frames, (0,0), (50, 17), (255,255,255), -1)
    cv2.putText(frames, fps, (3,10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0,0,0), 1)
    # Draw counts in top left corner
    cv2.rectangle(frames, (55,0), (165, 17), (255,255,255), -1)
    cv2.putText(frames, counts, (60,10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0,0,0), 1)
        
    # Display frames in a window  
    cv2.imshow('video2', frames) 
      
    # Wait for Esc key to stop 
    if cv2.waitKey(1) == ord('q'): 
        break
  
# De-allocate any associated memory usage 
cap.release()
cv2.destroyAllWindows() 


# Pedestrian detection (HOG+SVM) perdestrian2.mp4

In [34]:

from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os
import timer
import time
from timeit import default_timer as timer


# path setting
common_path = 'C:\\Users\\yukir\\anaconda3\\envs\\opencv\\Lib\\site-packages\\cv2\\data'
pedestrianPath='C:\\Users\\yukir\\workspace\\opencv_work\\Vehicle-And-Pedestrian-Detection-Using-Haar-Cascades\\Main Project\\Main Project\\Pedestrian Detection'

movie_path = os.path.join(pedestrianPath,"pedestrian2.mp4")



#shrink the rectangle from default
def draw_detections(img, rects, thickness = 2):
    for x, y, w, h in rects:
        pad_w, pad_h = int(0.15 * w), int(0.05 * h)
        cv2.rectangle(img, (x + pad_w, y + pad_h), (x + w - pad_w, y + h - pad_h), (0, 255, 0), thickness)
        
        text = "person" 
        text_top = (x, y-10) 
        text_bot = (x + 50, y + 5)
        text_pos = (x + 5, y)
        cv2.rectangle(frames, text_top, text_bot,(255,255,255), -1)
        cv2.putText(frames, text, text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,0), 1)  


#default detector
def PD_default(filename):
    #cv2.imread(filename) #read image
    frames = filename
    # HoG特徴量の計算
    hog = cv2.HOGDescriptor()
    #hog = cv2.HOGDescriptor((48,96), (16,16), (8,8), (8,8), 9)
    
    # SVMによる人検出
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    #hog.setSVMDetector(cv2.HOGDescriptor_getDaimlerPeopleDetector())
    
    #hogParams = {'winStride': (8, 8), 'padding': (32, 32), 'scale': 1.05}
    hogParams = {'hitThreshold':0,'winStride': (8,8), 'padding': (0,0), 'scale': 1.2,'finalThreshold':2}
        
    # convert to gray scale of each frames 
    #gray = cv2.cvtColor(frames, cv2.COLOR_BGR2GRAY) 
    #mask = bgs.apply(frames)
    #bg = bgs.getBackgroundImage()  
  
    # Detects cars of different sizes in the input image 
    people,r = hog.detectMultiScale(frames, **hogParams) 
     
    # compute NMS    
    #rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in people])
    #pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)
    # To draw a rectangle
    for (x,y,w,h) in people: 
        cv2.rectangle(frames,(x,y),(x+w,y+h),(255,255,255),2) 
        
        text = "person" 
        text_top = (x, y-10) 
        text_bot = (x + 50, y + 5)
        text_pos = (x + 5, y)
        cv2.rectangle(frames, text_top, text_bot,(255,255,255), -1)
        cv2.putText(frames, text, text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,0), 1)  
                
    # Display frames in a window  
    cv2.imshow('video2', frames) 
      
        
        
# capture frames from a video 
cap = cv2.VideoCapture(movie_path) 
start_frame=0
#bgs = cv2.createBackgroundSubtractorKNN()

if not cap.isOpened():
    raise IOError(("Couldn't open video file or webcam. If you're "
            "trying to open a webcam, make sure you video_path is an integer!"))
    
  
# Skip frames until reaching start_frame
if start_frame > 0:
        cap.set(cv2.CAP_PROP_POS_MSEC, start_frame)   
    
    
    
# loop runs if capturing has been initialized. 
count_num=0
while True: 
    # reads frames from a video 
    ret, frames = cap.read() 
    PD_default(frames)      
    
    # Wait for Esc key to stop 
    if cv2.waitKey(1) == ord('q'): 
        break
        
# De-allocate any associated memory usage 
cap.release()
cv2.destroyAllWindows() 
